In [1]:
import pandas as pd
import jellyfish as jf

In [2]:
rs = pd.read_table('SF-Rest-SmallSample.tsv', header=0)

In [3]:
rs.shape

(50630, 15)

In [31]:
rs.business_address = rs.business_address.str.lower()
rs.business_name = rs.business_name.str.lower()

In [32]:
small = rs.head(100)

In [33]:
rs.style
rs.head(50)

,business_name,business_address,business_city,business_state,business_postal_code,business_latitude,business_longitude,business_phone_number,inspection_id,inspection_date,inspection_score,inspection_type,violation_id,violation_description,risk_category
0,dar bar pakistani/indian cusine,1412 polk st,San Francisco,CA,94109,37.789784,-122.420455,NaN,1757_20170928,2017-09-28,86.0,Routine - Unscheduled,1757_20170928_103131,Moderate risk vermin infestation,Moderate Risk
1,dragon city bakery & cafe,2367 mission st,San Francisco,CA,94110,37.759174,-122.419066,1.415583e+10,4864_20161206,2017-12-06,84.0,Routine - Unscheduled,4864_20161206_103157,Food safety certificate or food handler card n...,Low Risk
2,deli 23,2449 23rd st,San Francisco,CA,94110,NaN,NaN,NaN,79782_20160503,2017-05-03,92.0,Routine - Unscheduled,79782_20160503_103120,Moderate risk food holding temperature,Moderate Risk
3,l'acajou bakery and cafe,498 09th st ste. c,San Francisco,CA,94103,NaN,NaN,NaN,73840_20171207,2017-12-07,71.0,Routine - Unscheduled,73840_20171207_103105,Improper cooling methods,High Risk
4,sweetheart cafe,909 grant ave,San Francisco,CA,94108,NaN,NaN,NaN,76437_20160329,2017-03-29,76.0,Routine - Unscheduled,76437_20160329_103113,Sewage or wastewater contamination,High Risk
5,cafe new honolulu,888 stockton st,San Francisco,CA,94108,NaN,NaN,NaN,82123_20171207,2017-12-07,87.0,Routine - Unscheduled,82123_20171207_103131,Moderate risk vermin infestation,Moderate Risk
6,aux delices vietnam,2327 polk st,San Francisco,CA,94109,37.798274,-122.422264,NaN,1146_20180223,2017-02-23,77.0,Routine - Unscheduled,1146_20180223_103103,High risk food holding temperature,High Risk
7,cafe picaro,3120 16th st,San Francisco,CA,94103,37.764908,-122.422442,1.415543e+10,3838_20180222,2017-02-22,72.0,Routine - Unscheduled,3838_20180222_103142,Unclean nonfood contact surfaces,Low Risk
8,jelly donut,3198 24th st,San Francisco,CA,94110,37.752439,-122.416250,NaN,38929_20160114,2017-01-14,88.0,Routine - Unscheduled,38929_20160114_103139,Improper food storage,Low Risk
9,limon peruvian rotisserie,1001 south van ness ave,San Francisco,CA,94110,NaN,NaN,1.415555e+10,81161_20160325,2017-03-25,92.0,Routine - Unscheduled,81161_20160325_103149,Wiping cloths not clean or properly stored or ...,Low Risk


In [34]:
def isNaN(x):
    return x != x

In [35]:
def block_on_postal_code(df):
    matches = {}
    pcs = (df.business_postal_code.unique())
    for pc in pcs:
        pc_df = df[df['business_postal_code'] == pc]
        matches = run_loop_df(pc_df, matches)
    return matches

In [40]:
def run_loop_df(df, matches):
    for idx, r in df.iterrows():
        for idk, j in df.iterrows():
            s = get_score_df(r,j)
            if s:
                print("MATCH!")
                print(r['business_name'])
                print(j['business_name'])
                already_in = matches.keys()
                if r.business_name not in already_in and j.business_name not in already_in:
                    matches[r['business_name']] = j['business_name']
    return matches

In [41]:
def get_score_df(i, j):
    
    i = list(str(i.values).split())
    j = list(str(j.values).split())
    
    if i[0] == j[0]:
        return False


    b = jf.jaro_winkler(i[0], j[0])
    a = jf.jaro_winkler(i[1], j[1])
    pc = jf.jaro_winkler(i[4], j[4])
    
    if isNaN(i[7]) or isNaN(j[7]):   
        score = b*0.45 + a*0.5 + pc*0.05
    else:
        ph_n = jf.jaro_winkler(i[7], j[7])
        score = b*0.3 + a*0.4 + ph_n*0.2 + pc*0.1
    


    if score > 0.8:
        return True
    return False


In [42]:
matches = block_on_postal_code(small)

MATCH!
cafe picaro
jade cafe
MATCH!
jade cafe
cafe picaro


In [43]:
matches

{'cafe picaro': 'jade cafe'}

In [44]:
rev = {v: k for k, v in matches.items()}

In [45]:
rev

{'jade cafe': 'cafe picaro'}

In [46]:
t = small.copy()

In [47]:
t

,business_name,business_address,business_city,business_state,business_postal_code,business_latitude,business_longitude,business_phone_number,inspection_id,inspection_date,inspection_score,inspection_type,violation_id,violation_description,risk_category
0,dar bar pakistani/indian cusine,1412 polk st,San Francisco,CA,94109,37.789784,-122.420455,NaN,1757_20170928,2017-09-28,86.0,Routine - Unscheduled,1757_20170928_103131,Moderate risk vermin infestation,Moderate Risk
1,dragon city bakery & cafe,2367 mission st,San Francisco,CA,94110,37.759174,-122.419066,1.415583e+10,4864_20161206,2017-12-06,84.0,Routine - Unscheduled,4864_20161206_103157,Food safety certificate or food handler card n...,Low Risk
2,deli 23,2449 23rd st,San Francisco,CA,94110,NaN,NaN,NaN,79782_20160503,2017-05-03,92.0,Routine - Unscheduled,79782_20160503_103120,Moderate risk food holding temperature,Moderate Risk
3,l'acajou bakery and cafe,498 09th st ste. c,San Francisco,CA,94103,NaN,NaN,NaN,73840_20171207,2017-12-07,71.0,Routine - Unscheduled,73840_20171207_103105,Improper cooling methods,High Risk
4,sweetheart cafe,909 grant ave,San Francisco,CA,94108,NaN,NaN,NaN,76437_20160329,2017-03-29,76.0,Routine - Unscheduled,76437_20160329_103113,Sewage or wastewater contamination,High Risk
5,cafe new honolulu,888 stockton st,San Francisco,CA,94108,NaN,NaN,NaN,82123_20171207,2017-12-07,87.0,Routine - Unscheduled,82123_20171207_103131,Moderate risk vermin infestation,Moderate Risk
6,aux delices vietnam,2327 polk st,San Francisco,CA,94109,37.798274,-122.422264,NaN,1146_20180223,2017-02-23,77.0,Routine - Unscheduled,1146_20180223_103103,High risk food holding temperature,High Risk
7,cafe picaro,3120 16th st,San Francisco,CA,94103,37.764908,-122.422442,1.415543e+10,3838_20180222,2017-02-22,72.0,Routine - Unscheduled,3838_20180222_103142,Unclean nonfood contact surfaces,Low Risk
8,jelly donut,3198 24th st,San Francisco,CA,94110,37.752439,-122.416250,NaN,38929_20160114,2017-01-14,88.0,Routine - Unscheduled,38929_20160114_103139,Improper food storage,Low Risk
9,limon peruvian rotisserie,1001 south van ness ave,San Francisco,CA,94110,NaN,NaN,1.415555e+10,81161_20160325,2017-03-25,92.0,Routine - Unscheduled,81161_20160325_103149,Wiping cloths not clean or properly stored or ...,Low Risk


In [49]:
for idx, r in t.iterrows():
    if r.business_name in matches.values():
        t = t.replace(r.business_name, rev[r.business_name])

In [50]:
t[t.business_name == 'jade cafe']

,business_name,business_address,business_city,business_state,business_postal_code,business_latitude,business_longitude,business_phone_number,inspection_id,inspection_date,inspection_score,inspection_type,violation_id,violation_description,risk_category


In [51]:
cleaninspection = t.copy()